In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import datetime
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
import sqlite3 as sq
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

Monthly Unemployment:  

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "LAUS" dataset, monthly  
+ Select the following dimensions: Regions, Year/Month   
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" (not available for places)   
+ Under "Year" choose your range (most recently I've selected 1990 - recent just to get it all)   
+ Select the value Labor Force (Seasonally Adjusted), Employed (Seasonally Adjusted)" , and Unemployed (Seasonally Adjusted). Unemployment rate is available but we're calculating it here in case we ever want to aggregate anything before calculating a percentage. It's been checked that this is correct and aligns with JobsEQ's data.   
+ Swap rows & columns if needed so "Regions" data is at the top, and "Year/Month" is at the side.  

When you've downloaded this csv, open it and delete the top line, make "Time" a double header, unmerge the regions and make sure they're copy-pasted across the relevant columns and keep the headers for different datapoints. Delete the data notes at the bottom, and select all of the numerical data and make them "General" format. Save as: 'JobsEQ_LAUSMonthly_GeosExceptPlaces.csv' 

In [29]:
#import monthly employment data and examine, use dtype str to import dates correctly
data = pd.read_csv('../../Data Downloads/JobsEQ_LAUSMonthly_GeosExceptPlaces.csv', dtype = str)
data.head(3)

,Time,USA,USA.1,USA.2,Tennessee (47),Tennessee (47).1,Tennessee (47).2,GNRC,GNRC.1,GNRC.2,GNRC Region,GNRC Region.1,GNRC Region.2,"Cheatham County, Tennessee (47021)","Cheatham County, Tennessee (47021).1","Cheatham County, Tennessee (47021).2","Davidson County, Tennessee (47037)","Davidson County, Tennessee (47037).1","Davidson County, Tennessee (47037).2","Dickson County, Tennessee (47043)","Dickson County, Tennessee (47043).1","Dickson County, Tennessee (47043).2","Houston County, Tennessee (47083)","Houston County, Tennessee (47083).1","Houston County, Tennessee (47083).2","Humphreys County, Tennessee (47085)","Humphreys County, Tennessee (47085).1","Humphreys County, Tennessee (47085).2","Maury County, Tennessee (47119)","Maury County, Tennessee (47119).1","Maury County, Tennessee (47119).2","Montgomery County, Tennessee (47125)","Montgomery County, Tennessee (47125).1","Montgomery County, Tennessee (47125).2","Robertson County, Tennessee (47147)","Robertson County, Tennessee (47147).1","Robertson County, Tennessee (47147).2","Rutherford County, Tennessee (47149)","Rutherford County, Tennessee (47149).1","Rutherford County, Tennessee (47149).2","Stewart County, Tennessee (47161)","Stewart County, Tennessee (47161).1","Stewart County, Tennessee (47161).2","Sumner County, Tennessee (47165)","Sumner County, Tennessee (47165).1","Sumner County, Tennessee (47165).2","Trousdale County, Tennessee (47169)","Trousdale County, Tennessee (47169).1","Trousdale County, Tennessee (47169).2","Williamson County, Tennessee (47187)","Williamson County, Tennessee (47187).1","Williamson County, Tennessee (47187).2","Wilson County, Tennessee (47189)","Wilson County, Tennessee (47189).1","Wilson County, Tennessee (47189).2"
0,Time,Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj),Labor Force (Seas Adj),Employed (Seas Adj),Unemployed (Seas Adj)
1,January 1990,126030369.5,118986131,7089527.083,2372360.535,2250217.963,123717.544,576899.855,553404.3727,23900.18263,604072.3401,579139.7623,25401.76022,13987.45804,13478.58862,487.8173447,273496.9054,263822.1768,9796.258828,17093.82485,16057.76584,1035.888421,2877.517978,2639.711093,240.3118815,7275.181406,6591.286257,679.3774404,27172.20191,25736.19827,1497.626956,39636.01449,37795.41347,1842.220968,20941.12715,19889.10416,1067.32977,63026.95069,60767.85225,2324.298915,3997.318824,3686.057058,308.1902782,53750.34367,51254.62229,2558.224877,3064.918026,2893.294843,175.6192435,42272.11193,41050.63023,1218.347274,35909.87875,33884.86038,2044.413098
2,February 1990,126182174.9,119206618.9,7064311.14,2377297.669,2255642.22,124196.531,578694.3735,555299.1898,24040.86616,605923.7222,581092.3967,25594.23968,14028.61071,13522.72341,490.1371996,274202.5803,264618.2165,9819.724835,17123.43551,16110.11402,1028.050628,2907.481509,2649.85335,259.7681082,7338.810314,6578.312166,760.3263316,27229.22155,25795.76145,1517.209699,39738.23835,37835.32634,1924.846269,21010.953,19960.472,

In [30]:
#set index to the extraneous "Time" header and transpose, then rename to "Datapoint" and reset index without dropping
data = data.set_index('Time').transpose()
data = data.rename(columns = {'Time': 'Datapoint'}).reset_index(drop = False)

In [31]:
#make sure there's no head or tail on the strings
data['index'] = data['index'].str.strip()
data['Datapoint'] = data['Datapoint'].str.strip()

In [32]:
#split and expand the geography names on the "." (where there were multiple from duplicates because of how we imported it) and take what was before as the name
boop = data['index'].str.split(pat = ".", expand = True)
data['NAME'] = boop[0]

In [33]:
#drop the original column, set the index to NAME and transpose to rename, use custom module, then transpose back and reset index without dropping
data = data.drop(columns = 'index').set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [34]:
#go from wide to long format with .melt() naming the columns as single column "Time", keeping "Datapoint" and "NAME" as is, and calling the number "Value"
cols = data.columns
data = data.melt(var_name = 'Time', id_vars = ['NAME', 'Datapoint'], value_vars = cols, value_name = 'Value')
data.head(2)

,NAME,Datapoint,Time,Value
0,United States,Labor Force (Seas Adj),January 1990,126030369.5
1,United States,Employed (Seas Adj),January 1990,118986131


In [35]:
#set index and transpose on datapoint to rename the original categories from JobsEQ, then transpose and reset index again without dropping
data = data.set_index('Datapoint').transpose()
data = data.rename(columns = {'Labor Force (Seas Adj)': 'LF:Labor Force', 'Employed (Seas Adj)': 'LF:Employed', 
                              'Unemployed (Seas Adj)': 'LF:Unemployed'})
data = data.transpose().reset_index(drop = False)

In [36]:
#make sure the value is a float datatype
data['Value'] = data['Value'].astype(float)

In [37]:
#create a new column equal to the value
data['New Value'] = data['Value']
#use .loc to say that in instances where the datapoint is equal to unemployment rate, new value is equal to value times 100 to match percent formatting
data.loc[data['Datapoint'] == 'LF:Unemployment %', 'New Value'] = data['Value'] * 100

In [38]:
data = data.drop(columns = 'Value').rename(columns = {'New Value': 'Value'})
data.head()

,Datapoint,NAME,Time,Value
0,LF:Labor Force,United States,January 1990,1.260304e+08
1,LF:Employed,United States,January 1990,1.189861e+08
2,LF:Unemployed,United States,January 1990,7.089527e+06
3,LF:Labor Force,Tennessee,January 1990,2.372361e+06
4,LF:Employed,Tennessee,January 1990,2.250218e+06


In [39]:
# RUN THIS IS YOU'RE INCLUDING THE YEAR 2000
weirds = ['January 2000', 'February 2000', 'March 2000', 'April 2000', 'May 2000', 'June 2000', 'July 2000', 'August 2000', 'September 2000', 
          'Octover 2000', 'November 2000', 'December 2000']
twothousand = data.loc[data['Time'].isin(weirds)]
data = data.loc[~data['Time'].isin(weirds)]

In [40]:
data.tail()

,Datapoint,NAME,Time,Value
21757,LF:Employed,"Williamson County, Tennessee",July 2023,137461.570600
21758,LF:Unemployed,"Williamson County, Tennessee",July 2023,3502.228326
21759,LF:Labor Force,"Wilson County, Tennessee",July 2023,83652.131480
21760,LF:Employed,"Wilson County, Tennessee",July 2023,81464.528270
21761,LF:Unemployed,"Wilson County, Tennessee",July 2023,2360.343494


In [41]:
#format the time for datetime by splitting the string and creating a month and year column
boop = data['Time'].str.split(pat = " ", expand = True)
data['month'] = boop[0]
data['year'] = boop[1]

In [42]:
twothousand.head()

,Datapoint,NAME,Time,Value
6480,LF:Labor Force,United States,January 2000,1.435761e+08
6481,LF:Employed,United States,January 2000,1.375819e+08
6482,LF:Unemployed,United States,January 2000,6.026013e+06
6483,LF:Labor Force,Tennessee,January 2000,2.850777e+06
6484,LF:Employed,Tennessee,January 2000,2.738339e+06


In [44]:
# RUN THIS IS YOU'RE INCLUDING THE YEAR 2000
boop = twothousand['Time'].str.split(pat = " ", expand = True)
twothousand['month'] = boop[0]
twothousand['year'] = boop[1]

In [45]:
# RUN THIS IS YOU'RE INCLUDING THE YEAR 2000
df = data.append(twothousand)
data = df

C:\Users\jmccall\AppData\Local\Temp\ipykernel_22184\2624616270.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = data.append(twothousand)


In [46]:
#set the index to the month and transpose/rename before transposing back and resetting the index without dropping
data = data.set_index('month').transpose()
data = data.rename(columns = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 
                              'May': 5, 'June': 6, 'July': 7, 'August': 8, 
                              'September': 9, 'October': 10, 'November': 11, 'December': 12, 
                             'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 
                              'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 
                              'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12})
data = data.transpose().reset_index(drop = False)

In [47]:
data['year'].unique()

array(['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', '2023'], dtype=object)

In [48]:
#do the same for year
data = data.set_index('year').transpose()
data = data.rename(columns = {'90': 1990, '91': 1991, '92': 1992, '93': 1993, '94': 1994, '95': 1995, 
                              '96': 1996, '97': 1997, '98': 1998, '99': 1999, 
                              '00': 2000, '1': 2001, '2': 2002, '3': 2003, '4': 2004, '5': 2005, 
                              '6': 2006, '7': 2007, '8': 2008, '9': 2009, '10': 2010, '11': 2011, 
                              '12': 2012, '13': 2013, '14': 2014, '15': 2015, '16': 2016, '17': 2017, 
                              '18': 2018, '19': 2019, '20': 2020, '21': 2021, '22': 2022})
data = data.transpose().reset_index(drop = False)

In [49]:
#add a filler for datetime to assume that this refers to the first day of each month
data['day'] = 1

In [50]:
data.head(2)

,year,month,Datapoint,NAME,Time,Value,day
0,1990,1,LF:Labor Force,United States,January 1990,126030369.5,1
1,1990,1,LF:Employed,United States,January 1990,118986131.0,1


In [51]:
#set these to datatype integer to input to datetime
cols = ['month', 'year', 'day']
data[cols] = data[cols].astype(int)

In [52]:
#we only want years 2000 on
data = data.loc[data['year'] > 1999]

In [53]:
#create a time stamp with these
data['TimeStamp'] = pd.to_datetime(data[['year', 'month', 'day']])

In [54]:
data.head(2)

,year,month,Datapoint,NAME,Time,Value,day,TimeStamp
6480,2000,10,LF:Labor Force,United States,October 2000,143988565.4,1,2000-10-01
6481,2000,10,LF:Employed,United States,October 2000,138076514.8,1,2000-10-01


In [55]:
#drop the Time and day columns and check before exporting
data = data.drop(columns = ['Time', 'day', 'year', 'month'])

In [56]:
#long to wide
data = pd.pivot(data, index = ['NAME', 'TimeStamp'], columns = 'Datapoint', values = 'Value')

In [57]:
data['LF:Unemployment %'] = (data['LF:Unemployed']/data['LF:Labor Force']) * 100

In [58]:
data.head(2)

Datapoint                              LF:Employed LF:Labor Force LF:Unemployed LF:Unemployment %
NAME                       TimeStamp                                                             
Cheatham County, Tennessee 2000-01-01  19309.92792    19836.25869    505.383288          2.547775
                           2000-02-01   19204.0417    19744.07237    520.174379          2.634585

In [59]:
#make sure the index is reset
data.reset_index(drop = False, inplace = True)
data = data.rename_axis(None, axis=1)
data.head()

,NAME,TimeStamp,LF:Employed,LF:Labor Force,LF:Unemployed,LF:Unemployment %
0,"Cheatham County, Tennessee",2000-01-01,19309.92792,19836.25869,505.383288,2.547775
1,"Cheatham County, Tennessee",2000-02-01,19204.0417,19744.07237,520.174379,2.634585
2,"Cheatham County, Tennessee",2000-03-01,19117.38708,19657.56312,525.301103,2.67226
3,"Cheatham County, Tennessee",2000-04-01,19056.95196,19588.60151,524.307372,2.676594
4,"Cheatham County, Tennessee",2000-05-01,19014.87426,19536.56016,521.94769,2.671646


In [60]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ LAUS'

In [61]:
#final check
data.head()

,NAME,TimeStamp,LF:Employed,LF:Labor Force,LF:Unemployed,LF:Unemployment %,GEO_ID,Source
0,"Cheatham County, Tennessee",2000-01-01,19309.92792,19836.25869,505.383288,2.547775,0500000US47021,JobsEQ LAUS
1,"Cheatham County, Tennessee",2000-02-01,19204.0417,19744.07237,520.174379,2.634585,0500000US47021,JobsEQ LAUS
2,"Cheatham County, Tennessee",2000-03-01,19117.38708,19657.56312,525.301103,2.67226,0500000US47021,JobsEQ LAUS
3,"Cheatham County, Tennessee",2000-04-01,19056.95196,19588.60151,524.307372,2.676594,0500000US47021,JobsEQ LAUS
4,"Cheatham County, Tennessee",2000-05-01,19014.87426,19536.56016,521.94769,2.671646,0500000US47021,JobsEQ LAUS


In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094 entries, 0 to 5093
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   NAME               5094 non-null   object        
 1   TimeStamp          5094 non-null   datetime64[ns]
 2   LF:Employed        5094 non-null   object        
 3   LF:Labor Force     5094 non-null   object        
 4   LF:Unemployed      5094 non-null   object        
 5   LF:Unemployment %  5094 non-null   object        
 6   GEO_ID             4245 non-null   object        
 7   Source             5094 non-null   object        
dtypes: datetime64[ns](1), object(7)
memory usage: 318.5+ KB


In [63]:
#export to the SQLite database as only the JobsEQ monthly data
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JOBSEQ_LAUS_Monthly', conn, if_exists = 'replace', index = False)

5094